In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json

host = "127.0.0.1"
port = 9998

sc = SparkContext("local[2]", "DisplayLinesV2")
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 60)  # Set batch interval to 10 seconds for quicker feedback

def process_rdd(rdd):
    print("Processing RDD")
    if not rdd.isEmpty():
        data = rdd.collect()
        for record in data:
            print(json.loads(record))
print("Waiting for a messages...")
lines = ssc.socketTextStream(host, port)

lines.foreachRDD(lambda rdd: process_rdd(rdd))
lines.pprint()
ssc.start()
ssc.awaitTermination()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
import socket
import json

from pyspark import SparkContext
from pyspark.sql import SparkSession


sc = SparkContext("local[*]", "SocketStreaming")
sc.setLogLevel("ERROR")


spark = SparkSession.builder \
    .appName("SocketStreaming") \
    .getOrCreate()


host = "127.0.0.1"
port = 9998


s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((host, port))
print("Connected to the producer...")


data_list = []


def process_data(data):
    try:
        
        record = json.loads(data)
        #print(record)
        #print(type(record))
        
        
        data_list.append(record)
        
        
        df = spark.createDataFrame(data_list)
        
        
        json_rdd = df.toJSON()
        json_data = [json_obj for json_obj in json_rdd.collect()]
        
        
        with open("raw.json", "a") as output_file:
            for item in json_data:
                output_file.write(item + ",\n")
        
        
        data_list.clear()
        
        
        #df.show()
        
    except Exception as e:
        print("Error processing data:", e)


while True:
    data = s.recv(1024)
    if not data:
        break
    
    process_data(data.decode("utf-8"))


s.close()


spark.stop()